# Basic feature store example
This notebook illustrates how you can use Databricks Feature Store to create, store, and manage features to train ML models and make batch predictions, including with features whose value is only available at the time of prediction. In this example, the goal is to predict the wine quality using a ML model with a variety of static wine features and a realtime input.

This notebook shows how to:
- Create a feature store table and use it to build a training dataset for a machine learning model.
- Modify the feature table and use the updated table to create a new version of the model.
- Use the Databricks feature store UI to determine how features relate to models.
- Perform batch scoring using automatic feature lookup.



In [0]:
import pandas as pd

from pyspark.sql.functions import monotonically_increasing_id, expr, rand
import uuid

from databricks import feature_store
from databricks.feature_store import feature_table, FeatureLookup

import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

## Load dataset
The code in the following cell loads the dataset and does some minor data preparation: creates a unique ID for each observation and removes spaces from the column names. The unique ID column (`wine_id`) is the primary key of the feature table and is used to lookup features.

In [0]:
raw_data = spark.read.load("/databricks-datasets/wine-quality/winequality-red.csv",format="csv",sep=";",inferSchema="true",header="true" )

def addIdColumn(dataframe, id_column_name):
    """Add id column to dataframe"""
    columns = dataframe.columns
    new_df = dataframe.withColumn(id_column_name, monotonically_increasing_id())
    return new_df[[id_column_name] + columns]

def renameColumns(df):
    """Rename columns to be compatible with Feature Store"""
    renamed_df = df
    for column in df.columns:
        renamed_df = renamed_df.withColumnRenamed(column, column.replace(' ', '_'))
    return renamed_df

# Run functions
renamed_df = renameColumns(raw_data)
df = addIdColumn(renamed_df, 'wine_id')

# Drop target column ('quality') as it is not included in the feature table
features_df = df.drop('quality')
display(features_df)


wine_id,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.998,3.16,0.58,9.8
4,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
5,7.4,0.66,0.0,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4
6,7.9,0.6,0.06,1.6,0.069,15.0,59.0,0.9964,3.3,0.46,9.4
7,7.3,0.65,0.0,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5
9,7.5,0.5,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.8,10.5


## Create a new database

In [0]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS wine_db")

# Create a unique table name for each run. This prevents errors if you run the notebook multiple times.
table_name = f"wine_db_" + str(uuid.uuid4())[:6]
print(table_name)

wine_db_f8ca8d


## Create the feature table

The first step is to create a FeatureStoreClient.

In [0]:
fs = feature_store.FeatureStoreClient()

# You can get help in the notebook for feature store API functions:
# help(fs.<function_name>)

# For example:
# help(fs.create_table)

Create the feature table. For a complete API reference, see ([AWS](https://docs.databricks.com/machine-learning/feature-store/python-api.html)|[Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/feature-store/python-api)|[GCP](https://docs.gcp.databricks.com/machine-learning/feature-store/python-api.html)).

In [0]:
fs.create_table(
    name=table_name,
    primary_keys=["wine_id"],
    df=features_df,
    schema=features_df.schema,
    description="wine features"
)

2024/03/31 05:09:23 INFO databricks.ml_features._compute_client._compute_client: Created feature table 'hive_metastore.default.wine_db_f8ca8d'.


<FeatureTable: name='default.wine_db_f8ca8d', table_id='2ca1a0d94b464a68b0a4805eef70c7b6', description='wine features', primary_keys=['wine_id'], partition_columns=[], features=['alcohol',
 'chlorides',
 'citric_acid',
 'density',
 'fixed_acidity',
 'free_sulfur_dioxide',
 'pH',
 'residual_sugar',
 'sulphates',
 'total_sulfur_dioxide',
 'volatile_acidity',
 'wine_id'], creation_timestamp=1711861747744, online_stores=[], notebook_producers=[notebook_id: 1739198649774171
revision_id: 1711861762879
creation_timestamp: 1711861763528
creator_id: "andriod.nag.user@gmail.com"
notebook_workspace_id: 6203166110358701
feature_table_workspace_id: 6203166110358701
notebook_workspace_url: "https://adb-6203166110358701.1.azuredatabricks.net"
producer_action: CREATE
], job_producers=[], table_data_sources=[], path_data_sources=['dbfs:/databricks-datasets/wine-quality/winequality-red.csv'], custom_data_sources=[], timestamp_keys=[], tags={}>

You can also use `create_table` without providing a dataframe, and then later populate the feature table using `fs.write_table`. `fs.write_table` supports both `overwrite` and `merge` modes.

Example:

```
fs.create_table(
    name=table_name,
    primary_keys=["wine_id"],
    schema=features_df.schema,
    description="wine features"
)

fs.write_table(
    name=table_name,
    df=features_df,
    mode="overwrite"
)
```

## Feature Store UI

To view the Feature Store UI, you must be in the Machine Learning persona ([AWS](https://docs.databricks.com/workspace/index.html#use-the-sidebar)|[Azure](https://learn.microsoft.com/azure/databricks/workspace/index#use-the-sidebar)|[GCP](https://docs.gcp.databricks.com/workspace/index.html#use-the-sidebar)). To access the Feature Store UI, click the Feature Store icon on the left navigation bar:  <img src="https://docs.databricks.com/_static/images/icons/feature-store-icon.png"/>  

The Databricks Feature Store UI shows a list of all feature tables in the workspace and also displays information about the tables including the creator, data sources, online stores, scheduled jobs that update the table, and the time the table was most recently updated. 

<img src="https://docs.databricks.com/_static/images/machine-learning/feature-store/basic-fs-nb-ui.png"/>  

Find the table you just created in the list. You can enter text into the Search box to search based on the name of a feature table, a feature, or a data source.

Click the table name to display details about the table. This page shows information about the table. The **Producers** section shows the name of the notebook that created the table and the last time the notebook was run. You can scroll down to the **Features** table for more details about the features in the table; an example is shown later in this notebook.

<img src="https://docs.databricks.com/_static/images/machine-learning/feature-store/basic-fs-nb-producers.png"/> 



## Train a model with feature store

The feature table does not include the prediction target. However, the training dataset needs the prediction target values. There may also be features that are not available until the time the model is used for inference. 

This example uses the feature **`real_time_measurement`** to represent a characteristic of the wine that can only be observed at inference time. This feature is used in training and the feature value for a wine is provided at inference time.

In [0]:
## inference_data_df includes wine_id (primary key), quality (prediction target), and a real time feature
inference_data_df = df.select("wine_id", "quality", (10 * rand()).alias("real_time_measurement"))
display(inference_data_df)

wine_id,quality,real_time_measurement
0,5,7.618387574233035
1,5,3.0775762143271446
2,5,9.4753525450113
3,6,7.871997487723817
4,5,3.4164492240137143
5,5,1.1084880893659832
6,5,7.727280549587237
7,7,9.479703303809277
8,7,2.579668117712597
9,5,1.5056577717898045


Use a `FeatureLookup` to build a training dataset that uses the specified `lookup_key` to lookup features from the feature table and the online feature `real_time_measurement`. If you do not specify the `feature_names` parameter, all features except the primary key are returned.

In [0]:
def load_data(table_name, lookup_key):
    # In the FeatureLookup, if you do not provide the `feature_names` parameter, all features except primary keys are returned
    model_feature_lookups = [FeatureLookup(table_name=table_name, lookup_key=lookup_key)]

    # fs.create_training_set looks up features in model_feature_lookups that match the primary key from inference_data_df
    training_set = fs.create_training_set(inference_data_df, model_feature_lookups, label="quality", exclude_columns="wine_id")
    training_pd = training_set.load_df().toPandas()

    # Create train and test datasets
    X = training_pd.drop("quality", axis=1)
    y = training_pd["quality"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, training_set

# Create the train and test datasets
X_train, X_test, y_train, y_test, training_set = load_data(table_name, "wine_id")
X_train.head()

,real_time_measurement,alcohol,chlorides,citric_acid,density,fixed_acidity,free_sulfur_dioxide,pH,residual_sugar,sulphates,total_sulfur_dioxide,volatile_acidity
493,0.939807,11.6,0.086,0.31,1.00020,8.7,23.0,3.48,3.0,0.74,81.0,0.690
354,8.892558,11.9,0.066,0.40,0.99120,6.1,40.5,3.25,1.4,0.59,165.0,0.210
342,2.635955,9.8,0.118,0.47,0.99820,10.9,6.0,3.30,1.8,0.75,14.0,0.390
834,5.613643,9.4,0.088,0.26,0.99694,8.8,16.0,3.32,1.6,0.47,23.0,0.685
705,6.035163,9.9,0.073,0.15,0.99900,8.4,11.0,3.37,6.0,0.49,54.0,1.035


In [0]:
from mlflow.tracking.client import MlflowClient

client = MlflowClient()

try:
    client.delete_registered_model("wine_model") # Delete the model if already created
except:
    None

The code in the next cell trains a scikit-learn RandomForestRegressor model and logs the model with the Feature Store.  

The code starts an MLflow experiment to track training parameters and results. Note that model autologging is disabled (`mlflow.sklearn.autolog(log_models=False)`); this is because the model is logged using `fs.log_model`.

In [0]:
# Disable MLflow autologging and instead log the model using Feature Store
mlflow.sklearn.autolog(log_models=False)

def train_model(X_train, X_test, y_train, y_test, training_set, fs):
    ## fit and log model
    with mlflow.start_run() as run:

        rf = RandomForestRegressor(max_depth=3, n_estimators=20, random_state=42)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)
 
        mlflow.log_metric("test_mse", mean_squared_error(y_test, y_pred))
        mlflow.log_metric("test_r2_score", r2_score(y_test, y_pred))

        fs.log_model(
            model=rf,
            artifact_path="wine_quality_prediction",
            flavor=mlflow.sklearn,
            training_set=training_set,
            registered_model_name="wine_model",
        )

train_model(X_train, X_test, y_train, y_test, training_set, fs)

2024/03/31 05:09:38 WARNING databricks.ml_features._training_scoring_client._training_scoring_client: Model could not be logged with a signature because the training set uses feature tables in Hive Metastore. Migrate the feature tables to Unity Catalog for model to be logged with a signature. See https://docs.databricks.com/en/machine-learning/feature-store/uc/upgrade-feature-table-to-uc.html for more details.
/databricks/python/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/03/31 05:09:42 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/10 [00:00<?, ?it/s]

2024/03/31 05:09:42 INFO mlflow.store.artifact.cloud_artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
Successfully registered model 'wine_model'.
2024/03/31 05:09:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: wine_model, version 1
Created version '1' of model 'wine_model'.



To view the logged model, navigate to the MLflow Experiments page for this notebook. To access the Experiments page, click the Experiments icon on the left navigation bar:  <img src="https://docs.databricks.com/_static/images/icons/experiments-icon.png"/>  

Find the notebook experiment in the list. It has the same name as the notebook, in this case, "Basic Feature Store Example".

Click the experiment name to display the experiment page. The packaged feature store model, created when you called `fs.log_model` appears in the **Artifacts** section of this page. You can use this model for batch scoring.  

<img src="https://docs.databricks.com/_static/images/machine-learning/feature-store/basic-fs-nb-artifact.png"/>  

The model is also automatically registered in the Model Registry. The feature table in the Feature Store UI is also updated to show which features from the feature table were used to train the model. 

## Batch scoring
Use `score_batch` to apply a packaged feature store model to new data for inference. The input data only needs the primary key column `wine_id` and the realtime feature `real_time_measurement`. The model automatically looks up all of the other feature values from the feature store.


In [0]:
## For simplicity, this example uses inference_data_df as input data for prediction
batch_input_df = inference_data_df.drop("quality") # Drop the label column

predictions_df = fs.score_batch("models:/wine_model/latest", batch_input_df)
                                  
display(predictions_df["wine_id", "prediction"])

/databricks/python/lib/python3.11/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.11.1/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


2024/03/31 05:09:51 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


2024/03/31 05:09:51 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.


2024/03/31 05:09:52 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


wine_id,prediction
0,5.209341384824892
1,5.463040444836221
2,5.463040444836221
3,5.472774758308718
4,5.209341384824892
5,5.218869115709728
6,5.138934654737697
7,4.942007363826638
8,5.237243683413905
9,5.578680452146556


## Modify feature table
Suppose you modify the dataframe by adding a new feature. You can use `fs.write_table` with `mode="merge"` to update the feature table.

In [0]:
## Modify the dataframe containing the features
so2_cols = ["free_sulfur_dioxide", "total_sulfur_dioxide"]
new_features_df = (features_df.withColumn("average_so2", expr("+".join(so2_cols)) / 2))

display(new_features_df)

wine_id,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,average_so2
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,22.5
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8,46.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,34.5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.998,3.16,0.58,9.8,38.5
4,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,22.5
5,7.4,0.66,0.0,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,26.5
6,7.9,0.6,0.06,1.6,0.069,15.0,59.0,0.9964,3.3,0.46,9.4,37.0
7,7.3,0.65,0.0,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,18.0
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,13.5
9,7.5,0.5,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.8,10.5,59.5


Update the feature table using `fs.write_table` with `mode="merge"`.

In [0]:
fs.write_table(
    name=table_name,
    df=new_features_df,
    mode="merge"
)

## Explore feature lineage in the UI

After you update the feature table, the UI reflects the following:
- The new feature appears in the feature list.
- Deleted features still appear. When you read in the feature table, the deleted features have null values.
- The **Models** column displays model versions that use each feature.
- The **Notebooks** column displays notebooks that use each feature.

<img src="https://docs.databricks.com/_static/images/machine-learning/feature-store/basic-fs-nb-feature-table.png"/>  

To read feature data from the Feature Store, use `fs.read_table()`.

In [0]:
# Displays most recent version of the feature table
# Note that features that were deleted in the current version still appear in the table but with value = null.
display(fs.read_table(name=table_name))

wine_id,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,average_so2
0,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,22.5
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.2,0.68,9.8,46.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,34.5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.998,3.16,0.58,9.8,38.5
4,7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,22.5
5,7.4,0.66,0.0,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,26.5
6,7.9,0.6,0.06,1.6,0.069,15.0,59.0,0.9964,3.3,0.46,9.4,37.0
7,7.3,0.65,0.0,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,18.0
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,13.5
9,7.5,0.5,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.8,10.5,59.5


## Train a new model version using the updated feature table

In [0]:
def load_data(table_name, lookup_key):
    model_feature_lookups = [FeatureLookup(table_name=table_name, lookup_key=lookup_key)]
    
    # fs.create_training_set will look up features in model_feature_lookups with matched key from inference_data_df
    training_set = fs.create_training_set(inference_data_df, model_feature_lookups, label="quality", exclude_columns="wine_id")
    training_pd = training_set.load_df().toPandas()

    # Create train and test datasets
    X = training_pd.drop("quality", axis=1)
    y = training_pd["quality"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, training_set

X_train, X_test, y_train, y_test, training_set = load_data(table_name, "wine_id")
X_train.head()

,real_time_measurement,alcohol,average_so2,chlorides,citric_acid,density,fixed_acidity,free_sulfur_dioxide,pH,residual_sugar,sulphates,total_sulfur_dioxide,volatile_acidity
493,0.939807,11.6,52.00,0.086,0.31,1.00020,8.7,23.0,3.48,3.0,0.74,81.0,0.690
354,8.892558,11.9,102.75,0.066,0.40,0.99120,6.1,40.5,3.25,1.4,0.59,165.0,0.210
342,2.635955,9.8,10.00,0.118,0.47,0.99820,10.9,6.0,3.30,1.8,0.75,14.0,0.390
834,5.613643,9.4,19.50,0.088,0.26,0.99694,8.8,16.0,3.32,1.6,0.47,23.0,0.685
705,6.035163,9.9,32.50,0.073,0.15,0.99900,8.4,11.0,3.37,6.0,0.49,54.0,1.035


Build a training dataset that will use the indicated `key` to lookup features.

In [0]:
def train_model(X_train, X_test, y_train, y_test, training_set, fs):
    ## fit and log model
    with mlflow.start_run() as run:

        rf = RandomForestRegressor(max_depth=3, n_estimators=20, random_state=42)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)

        mlflow.log_metric("test_mse", mean_squared_error(y_test, y_pred))
        mlflow.log_metric("test_r2_score", r2_score(y_test, y_pred))

        fs.log_model(
            model=rf,
            artifact_path="feature-store-model",
            flavor=mlflow.sklearn,
            training_set=training_set,
            registered_model_name="wine_model",
        )

train_model(X_train, X_test, y_train, y_test, training_set, fs)

2024/03/31 05:10:28 WARNING databricks.ml_features._training_scoring_client._training_scoring_client: Model could not be logged with a signature because the training set uses feature tables in Hive Metastore. Migrate the feature tables to Unity Catalog for model to be logged with a signature. See https://docs.databricks.com/en/machine-learning/feature-store/uc/upgrade-feature-table-to-uc.html for more details.
2024/03/31 05:10:32 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.11.1/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.


Uploading artifacts:   0%|          | 0/10 [00:00<?, ?it/s]

2024/03/31 05:10:32 INFO mlflow.store.artifact.cloud_artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
Registered model 'wine_model' already exists. Creating a new version of this model...
2024/03/31 05:10:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: wine_model, version 2
Created version '2' of model 'wine_model'.


Apply the latest version of the feature store registered MLflow model to features using **`score_batch`**.

In [0]:
## For simplicity, this example uses inference_data_df as input data for prediction
batch_input_df = inference_data_df.drop("quality") # Drop the label column
predictions_df = fs.score_batch(f"models:/wine_model/latest", batch_input_df)
display(predictions_df["wine_id","prediction"])

/databricks/python/lib/python3.11/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.11.1/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


2024/03/31 05:10:40 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


2024/03/31 05:10:41 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.


2024/03/31 05:10:41 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


wine_id,prediction
0,5.209341384824892
1,5.452837398542869
2,5.502254426489755
3,5.472591947615569
4,5.209341384824892
5,5.218869115709728
6,5.138934654737697
7,5.155432662453844
8,5.237243683413905
9,5.568477405853204


The UI now reflects that version 2 of `wine_model` uses the newly created feature `average_so2`.
  
<img src="https://docs.databricks.com/_static/images/machine-learning/feature-store/basic-fs-nb-new-feature.png"/>  

## Control permissions for and delete feature tables
- To control who has access to a feature table, use the **Permissions** button in the UI.
- To delete a feature table, click the kebab menu next to **Permissions** in the UI and select **Delete**. When you delete a feature table using the UI, the corresponding Delta table is not deleted; you must delete that table separately. 